In [14]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2021-02-05 05:18:00--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Résolution de files.grouplens.org (files.grouplens.org)… 128.101.65.152
Connexion à files.grouplens.org (files.grouplens.org)|128.101.65.152|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 978202 (955K) [application/zip]
Enregistre : «ml-latest-small.zip.1»

ml-latest-small.zip 100%[===================>] 955,28K  1,52MB/s    ds 0,6s    

2021-02-05 05:18:01 (1,52 MB/s) - «ml-latest-small.zip.1» enregistré [978202/978202]

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [39]:
csv_file=pd.read_csv("ml-latest-small/ratings.csv")
utility_matrix=csv_file.pivot(index='movieId', columns='userId', values='rating').fillna(0.0)
utility_matrix
 

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
R = utility_matrix.to_numpy()
#R=R[:,:]
print(R.shape[0])

9724


In [63]:
class SVD():
    def __init__(self, M):
        self.M = M
        
    def computeSVD(self,M):
        U, S, VT = np.linalg.svd(self.M, full_matrices=False)
        s=np.diag(S[0:2])
        return U, S, VT, s
        
    def plot_singular_values(self,s):
        plt.semilogy(np.diagonal(s))
        plt.title('plot Singular Values')
        plt.show()
        
    def rank_S(self,S):
        print("rank:")
        return len(S)
  

In [114]:
svd=SVD(R)
U, S, VT, s=svd.computeSVD(R)

#svd.plot_singular_values(s)


# calculate initial P and Q 
# we fill missing rating of the original matrix with zeros
# we calculate the svd of this matrix
# we do the svd of this matriw , we will not use the svd to predict missing values but only to factorize the 
# initial estimated matrix model
# R = U S VT
# lets make : Q=U, and pT=S * VT

# lets start : 

print("Initializing Q and pT \n")
Q=U[:,:2]
print("Q matrix:",Q.shape,"\n",Q)
pT=s@VT[:2,:]
print("\npT matrix:",pT.shape,"\n",pT)

Initializing Q and pT 

Q matrix: (9724, 2) 
 [[-7.04498985e-02 -2.75911949e-02]
 [-3.85393459e-02 -2.06662722e-03]
 [-1.59129220e-02 -2.47146155e-02]
 ...
 [-6.46836073e-05  5.97586244e-04]
 [-6.46836073e-05  5.97586244e-04]
 [-2.71729303e-04  1.27236200e-03]]

pT matrix: (2, 610) 
 [[-29.68924408  -3.13506492  -0.72319333 ... -62.05385826  -4.05060109
  -74.2121552 ]
 [-14.26125156   4.10160993  -0.47833901 ...  -2.73946973  -3.18751205
   46.75244689]]


In [113]:
#getting non zero values from the original matrix:

len_items, len_users = R.shape

i_j_R= [ 
            (i, j, R[i, j])
            for i in range(len_items)
            for j in range(len_users)
            if R[i, j] != 0
        ]


In [106]:
# Perform SGD 

grad_Q=0 #np.zeros((len_items,K))

n=0.1
lamda=0.1

Q_=Q
pT_=pT

for _ in range(10): 
    
    for i_ in range(R.shape[0]):
        for k in range(2):
            for x in range(R.shape[1]):
                grad_Q+=-2*(R[i_,x]-(Q[i_,:]@pT[:,x]))*pT[k,x] + 2*lamda*Q[i_,k]
            Q_[i_,k]-=n*grad_Q


        

           
   

In [112]:
print(Q_.shape)
print(Q_)

(50, 2)
[[ 4.18852218e+38  4.29557582e+38]
 [ 4.90787135e+38  5.03105068e+38]
 [ 5.73403221e+38  5.87540004e+38]
 [ 6.68042596e+38  6.84225138e+38]
 [ 7.76170306e+38  7.94645503e+38]
 [ 8.99359255e+38  9.20390681e+38]
 [ 1.03925079e+39  1.06311078e+39]
 [ 1.19747579e+39  1.22443022e+39]
 [ 1.37551569e+39  1.40579771e+39]
 [ 1.57447616e+39  1.60824408e+39]
 [ 1.79473858e+39  1.83201187e+39]
 [ 2.03544551e+39  2.07601107e+39]
 [ 2.29376620e+39  2.33704559e+39]
 [ 2.56387626e+39  2.60874256e+39]
 [ 2.83557322e+39  2.88010372e+39]
 [ 3.09243449e+39  3.13358339e+39]
 [ 3.30940913e+39  3.34258134e+39]
 [ 3.44971657e+39  3.46822102e+39]
 [ 3.46090709e+39  3.45526458e+39]
 [ 3.26991755e+39  3.22699482e+39]
 [ 2.77693375e+39  2.67887151e+39]
 [ 1.84784641e+39  1.67074512e+39]
 [ 3.05061645e+38  1.73843762e+37]
 [-2.08360132e+39 -2.52295427e+39]
 [-5.61822221e+39 -6.26221739e+39]
 [-1.06826308e+40 -1.15988400e+40]
 [-1.77631598e+40 -1.90369943e+40]
 [-2.74706997e+40 -2.92092078e+40]
 [-4.0566491

In [109]:
grad_pT=0 #np.zeros((K,len_users))

for _ in range(10): 
    
    for k_ in range(2):
        for x in range(R.shape[1]):
            for i in range(R.shape[0]):
                grad_pT+=-2*(R[i,x]-(Q[i,:]@pT[:,x]))*Q[i,k_] + 2*lamda*pT[k_,x]
            pT_[k_,x]-=n*grad_pT

<ipython-input-109-e490f42bb9b7>:8: RuntimeWarning: overflow encountered in double_scalars
  grad_pT+=-2*(R[i,x]-(Q[i,:]@pT[:,x]))*Q[i,k_] + 2*lamda*pT[k_,x]
<ipython-input-109-e490f42bb9b7>:8: RuntimeWarning: invalid value encountered in double_scalars
  grad_pT+=-2*(R[i,x]-(Q[i,:]@pT[:,x]))*Q[i,k_] + 2*lamda*pT[k_,x]


In [110]:
print(pT_)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
